In [1]:
def log_progress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

In [2]:
import matplotlib.pylab as plt
import seaborn as sns
sns.despine()

import numpy as np
 
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger, EarlyStopping
from keras.optimizers import RMSprop, Adam, SGD, Nadam
from keras.layers.advanced_activations import *
from keras.layers import Convolution1D, MaxPooling1D, AtrousConvolution1D
from keras.layers.recurrent import LSTM, GRU
from keras import regularizers

# import theano
# theano.config.compute_test_value = "ignore"

Using TensorFlow backend.
/Users/stas/Documents/NTC/crm-master/crm/env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/stas/Documents/NTC/crm-master/crm/env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/stas/Documents/NTC/crm-master/crm/env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.in

In [ ]:
num_time_step = 20
forecast_num_time_step = 1
time_series_len = 10000
batch_size = 10
time_series = [np.sin(x) for x in np.linspace(0, 50, time_series_len)]

# plt.figure(figsize=(15,10))
# plt.plot(time_series)
# plt.show()

X_train = []
y_train = []
for i in range(len(time_series[num_time_step:-forecast_num_time_step])):
    X_train.append(time_series[i:i+num_time_step])
    y_train.append(time_series[i+num_time_step:i+num_time_step+forecast_num_time_step])

X_train = np.array(X_train).reshape((time_series_len - num_time_step - forecast_num_time_step,num_time_step))
y_train = np.array(y_train)

model = Sequential()
model.add(Dense(512,input_shape=(num_time_step,)))
model.add(LeakyReLU())
model.add(Dropout(0.5))
model.add(Dense(256))
model.add(LeakyReLU())

model.add(Dropout(0.5))
model.add(Dense(128))
model.add(LeakyReLU())

model.add(Dropout(0.5))
model.add(Dense(64))
model.add(LeakyReLU())

model.add(Dropout(0.5))
model.add(Dense(32))
model.add(LeakyReLU())

model.add(Dropout(0.5))
model.add(Dense(forecast_num_time_step, activation='linear'))

model.compile(optimizer=Adam(), 
              loss='mean_squared_error')

In [ ]:
history = model.fit(X_train, y_train, 
          epochs = 10, 
          batch_size = batch_size, 
          verbose=1, 
#           validation_data=(X_train, y_train),
#           callbacks=[reduce_lr, checkpointer],
          shuffle=False)

In [ ]:
%%time 
X_test = []
y_test = []
for i in log_progress(range(len(time_series[num_time_step:-forecast_num_time_step]))):
    X_test.append(model.predict(np.array(time_series[i:i+num_time_step]).reshape((-1,num_time_step))))

X_test = np.array(X_test)

# for i in log_progress(range(1000)):
#     np.append(X_test,model.predict(X_test[-1])

In [ ]:
print(X_test.shape)
for i in log_progress(range(100)):
    X_test = np.append(X_test,model.predict(X_test[-num_time_step:].reshape((-1,num_time_step))))
print(X_test.shape)

In [ ]:
plt.figure(figsize=(15,10))
plt.plot(X_test.flatten())
plt.plot(time_series[num_time_step:-forecast_num_time_step])
plt.legend(['pred','true'], loc='best')
plt.show()